## RNN 실습
- RNN MNIST 분류기
- RNN 주가예측(Regression)
- RNN 문장만들어내기 (charRNN)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Junhojuno/DeepLearning-Beginning/blob/master/9-2.RNN_LSTM.ipynb)

### 1. MNIST Classifier
- 28x28 image에서 28개의 벡터가 28번 순차적으로 들어간다고 하고 마지막 28번째 벡터가 들어오면 분류가 되는 형태로 진행

In [6]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import tensorflow.contrib.rnn as rnn
mnist = input_data.read_data_sets("data/", one_hot=True)
print("PACKAGE LOADED")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
PACKAGE LOADED


In [0]:
trainimgs, trainlabels, testimgs, testlabels \
    = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

In [8]:
ntrain = trainimgs.shape[0]
ntest = testimgs.shape[0]
dim = trainimgs.shape[1]
nclasses = trainlabels.shape[1]

print("MNIST LOADED.")
print("TF VERSION : %s" % (tf.__version__))

MNIST LOADED.
TF VERSION : 1.12.0-rc2


**아래 그림과 같은 형태로 진행**

![](https://github.com/Junhojuno/DeepLearning-Beginning/blob/master/NN_images/process.jpg?raw=1)

### Define Model

In [0]:
diminput = 28
dimhidden = 128
dimoutput = nclasses
nsteps = 28
weights = {
   
    # 28 dimension에서 128 dimension으로,
    # 128 dimension에서 10 dimension으로 가는 weights를 잡아준다.
    
    'hidden': tf.Variable(initial_value=tf.random_normal([diminput, dimhidden])), 
    'out': tf.Variable(initial_value=tf.random_normal([dimhidden,dimoutput])),
}
biases = {
    'hidden': tf.Variable(initial_value=tf.random_normal([dimhidden])),
    'out': tf.Variable(initial_value=tf.random_normal([dimoutput])),
}

### Define Function

In [10]:
import numpy as np
a = np.array([[[0,0,0],[1,1,1],[2,2,2],[3,3,3]],
              [[4,4,4],[5,5,5],[6,6,6],[7,7,7]]])
print(a.shape)
a_hat = np.transpose(a, [1,0,2])
print(a_hat.shape)

a_hat = np.reshape(a_hat, (-1,3))
print(a_hat.shape)
a_split = np.split(a_hat, 4, axis=0)
print(a_split)

(2, 4, 3)
(4, 2, 3)
(8, 3)
[array([[0, 0, 0],
       [4, 4, 4]]), array([[1, 1, 1],
       [5, 5, 5]]), array([[2, 2, 2],
       [6, 6, 6]]), array([[3, 3, 3],
       [7, 7, 7]])]


![](https://github.com/Junhojuno/DeepLearning-Beginning/blob/master/NN_images/RNN.png?raw=1)

1. Cell을 정의한다.(BasicRNNCell,BasicLSTMCell...) (Cell 생성)
    - 위 그림에서 A 박스 부분이 Cell이라 보면 된다.
    - 여기서 num_units는 hidden_size를 말하는데, 출력($h_{t}$)의 크기를 말한다.
    

2. Cell을 구동시켜서 입력을 주고 출력값을 뽑아내는 드라이버를 만든다.(Cell 구동)
    - 드라이버로는 dynamic_rnn 등이 있고,
    - 드라이버 입력값은 정의한 cell, 입력데이터($x_{t}$)순서!
    - 이 드라이버의 출력값은 $h_{t}$와 state값이다.

In [11]:
def _RNN(_x, _istate, _w, _b, _nsteps, _name):
    # 1. batch_sizex28x28 --> 28xbatch_sizex28로 바꿔준다.
    _x = tf.transpose(_x, [1,0,2])
    # 2. Reshape input : [nstep*batch_size, diminput]
    _x = tf.reshape(_x, shape=[-1, diminput])
    
    # 3. Input to hidden layer
    # 140x28 input과 28x128 weights의 행렬곱
    # output : 140x128
    _H = tf.matmul(_x, _w['hidden']) + _b['hidden']
    # 4. Split Data to 'nstep(28)' chunks --> batch_size만큼 한묶음인 data의 List를 만들어주는 과정이다.
    _Hsplit = tf.split(_H, _nsteps, axis=0)
    
    # 5. Get LSTM's Final output (_LSTM_O) and state (_LSTM_S)
    with tf.variable_scope(_name):
        # rnn = tf.contrib.rnn
        # cell 정의
        # Gate에 들어있는 neural net의 weight를 알아서 선언해줌
        # 먼저 cell을 만들자(num_units는 output의 크기를 의미)
        lstm_cell = rnn.BasicLSTMCell(num_units=dimhidden, forget_bias=1.0
                                     , state_is_tuple=False)
        # cell을 만들었으니 구동시키자
        # parameter는 cell, 입력데이터 순서(x_t)이다.
        # sequence length : 28
        # 즉 A 박스가 28개가 있다는 얘기다.
        _LSTM_O, _LSTM_S = rnn.static_rnn(lstm_cell, _Hsplit, initial_state=_istate)
    
    # Output
    _o = tf.matmul(_LSTM_O[-1], _w['out']) + _b['out']
    return {
        'x': _x, 'H': _H, 'Hsplit': _Hsplit, 'LSTM_O': _LSTM_O, 'LSTM_S': _LSTM_S, 
        'o': _o 
    }
print("Function Ready..!!")

Function Ready..!!


### Define Graph

In [12]:
learning_rate = 0.001
x = tf.placeholder('float', [None, nsteps, diminput])
# state & cell = 2 * n_hidden
istate = tf.placeholder('float', [None, 2*dimhidden]) # istate means initial state
y = tf.placeholder('float', [None, dimoutput])

myrnn = _RNN(x, istate, weights, biases, _nsteps=nsteps, _name='basic')
pred = myrnn['o']

# Cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1), tf.argmax(y,1)), tf.float32))
init = tf.global_variables_initializer()
print("Network Ready..!!")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Network Ready..!!


### Run

In [0]:
training_epochs = 5
batch_size = 100
display_step = 1
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(init)

**GPU가 없어서리...실행불가...**

### Optimizer

In [15]:
print ("START OPTIMIZATION.")
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, nsteps, diminput))
        # Fit training using batch data
        feeds = {x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
        sess.run(optimize, feed_dict=feeds)
        # compute average loss
        avg_cost += sess.run(cost, feed_dict=feeds)/total_batch
    # Display logs per epoch step
    if epoch % display_step == 0:
        print("EPOCH: %03d/%03d COST: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x:batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
        train_acc = sess.run(accuracy, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        testimgs = testimgs.reshape((ntest, nsteps, diminput))
        feeds = {x: testimgs, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))}
        test_acc = sess.run(accuracy, feed_dict=feeds)
        print (" TEST ACCURACY : %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED.")

START OPTIMIZATION.
EPOCH: 000/005 COST: 0.431314853
TRAIN ACCURACY: 0.910
 TEST ACCURACY : 0.942
EPOCH: 001/005 COST: 0.116124308
TRAIN ACCURACY: 0.990
 TEST ACCURACY : 0.961
EPOCH: 002/005 COST: 0.075874198
TRAIN ACCURACY: 0.990
 TEST ACCURACY : 0.970
EPOCH: 003/005 COST: 0.053807406
TRAIN ACCURACY: 1.000
 TEST ACCURACY : 0.973
EPOCH: 004/005 COST: 0.045213481
TRAIN ACCURACY: 0.990
 TEST ACCURACY : 0.978
OPTIMIZATION FINISHED.
